In [18]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import tqdm
import datetime
from collections import Counter
import matplotlib.pyplot as plt

df_usage = pd.read_csv('data/App_usage_trace.txt', sep=' ', names=['user', 'time', 'location', 'app', 'traffic'])
df_usage2 = df_usage[['user', 'time', 'app']]
df_loc = df_usage['location']
df_usage2.head()

# merging the consecutive usage records of the same app in one minute
df_usage2['time'] = df_usage2['time'].apply(lambda x : str(x)[:-2])
df_usage2.drop_duplicates(inplace=True)

# delete app used less than 10 times for all users
df_usage2 = df_usage2[df_usage2.groupby('app')['app'].transform('count').ge(10)]

df_usage = df_usage2.join(df_loc)

C:\Users\69482\AppData\Local\Temp\ipykernel_7216\3956847666.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_usage2['time'] = df_usage2['time'].apply(lambda x : str(x)[:-2])
C:\Users\69482\AppData\Local\Temp\ipykernel_7216\3956847666.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_usage2.drop_duplicates(inplace=True)


In [19]:
prev_user = -1
prev_time = -1
app_seq = []
time_seq = []
all_app_seq = []
all_time_seq = []

seq_length = 3

# df_usage is already sorted based on user and time sequence
for i in tqdm.tqdm(range(len(df_usage))):
    user = df_usage.iloc[i]['user']
    app = df_usage.iloc[i]['app']
    loc = df_usage.iloc[i]['location']
    time = df_usage.iloc[i]['time']
    time = datetime.datetime.strptime(time, '%Y%m%d%H%M')

    if prev_user != user:
        app_seq = [app]
        time_seq = [time]
        all_app_seq.append([])
        all_time_seq.append([])

    else:
        # same sequence if the time gap between them is equal to or less than 7 mins
        if (time - prev_time).total_seconds() // 60 <= 7:
            if len(app_seq) == seq_length:
                all_app_seq.append(app_seq)
                # 差分时间
                all_time_seq.append([(prev_time - x).total_seconds() // 60 for x in time_seq])
                app_seq = app_seq[1:] + [app]
                time_seq = time_seq[1:] + [time]
            else:
                app_seq.append(app)
                time_seq.append(time)
                all_app_seq.append([])
                all_time_seq.append([])
        else:
            app_seq = [app]
            time_seq = [time]
            all_app_seq.append([])
            all_time_seq.append([])
        
    prev_user = user
    prev_time = time

100%|██████████| 1123955/1123955 [09:22<00:00, 1996.95it/s]


In [20]:
df_usage['app_seq'] = all_app_seq
df_usage['time_seq'] = all_time_seq

# only filled sequences are treated as data
df_usage = df_usage[df_usage['app_seq'].map(len) != 0]
# delete users who have sequences less than 50
df_usage = df_usage[df_usage.groupby('user')['user'].transform('count').ge(50)]

df_usage.head()

,user,time,app,location,app_seq,time_seq
7,0,201604200816,360,8194,"[361, 361, 31]","[3.0, 0.0, 0.0]"
8,0,201604200816,612,8194,"[361, 31, 360]","[0.0, 0.0, 0.0]"
10,0,201604200817,31,8194,"[31, 360, 612]","[0.0, 0.0, 0.0]"
13,0,201604200817,360,8194,"[360, 612, 31]","[1.0, 1.0, 0.0]"
14,0,201604200817,361,8194,"[612, 31, 360]","[1.0, 0.0, 0.0]"


In [21]:
df_usage.to_csv('data/baseline_loc_time_3.txt', sep='\t', index=False)